In [1]:
import pandas as pd

In [2]:
maple_train_df = pd.read_csv('maple_train.csv')
maple_test_df = pd.read_csv('maple_test.csv')
maple_eval_df = pd.read_csv('maple_eval.csv')

In [51]:
# maple_train_df[maple_train_df["Path"] == "nitrogen"]

In [16]:
import os
path = './PathwayFeatures'
pathway_names = []
pw_names = maple_eval_df["Path"].unique()
for i, attrlist in enumerate(os.listdir(path)):
    attributes = pd.read_csv(path+'/'+ attrlist)
    attributes = attributes["Attrib"].tolist()
    pathway_name = attrlist.split('-')[0]
    pathway_names.append(pathway_name)

    maple_train_df.loc[maple_train_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'
    maple_test_df.loc[maple_test_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'
    maple_eval_df.loc[maple_eval_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'

    maple_train_df.loc[maple_train_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'
    maple_test_df.loc[maple_test_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'
    maple_eval_df.loc[maple_eval_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'

    train_df_prep = maple_train_df[attributes]
    test_df_prep = maple_test_df[attributes]
    eval_df_prep = maple_eval_df[attributes]

    train_df_prep.to_csv('Datasets/'+pathway_name+'_train.csv', index=False)
    test_df_prep.to_csv('Datasets/'+pathway_name+'_test.csv', index=False)
    eval_df_prep.to_csv('Datasets/'+pathway_name+'_eval.csv', index=False)
    
    if (i == 1):
        break

In [17]:
pathway_names

['amine_and_polyamine', 'amino_acid']

In [18]:
# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
# by: @invoktheshell
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)):
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, recall_score, precision_score

def calc_metrics(y_true, y_pred, model_name, pw_name, dataset_name, fold_num):
    acc = accuracy_score(y_true, y_pred)
    tp, fp, tn, fn = perf_measure(y_true, y_pred)
    recall = precision_score(y_true, y_pred)
    specifity = tn / (tn + fp)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    # print(precision, specifity, tp,fp,tn,fn)
    
    try:
        fpr = fp / (fp + tn)
    except:
        fpr = 0
        
    try:
        tpr = tp / (tp + fn)
    except:
        tpr = 0
    
    try:
        fnr = fn / (fn + tp)
    except:
        fnr = 0
    
    try:
        fdr = fp / (fp + tp)
    except:
        fdr = 0
    
    return model_name, pw_name, dataset_name, fold_num, acc, f1, mcc, tp, fp, tn, fn, recall, precision, specifity, fpr, tpr, fnr, fdr

In [57]:
#.3 Metabolic pathway modeling and analysis
# The 10-fold cross-validation and bagging were used on all training process,
# and the metrics accuracy, percentage of correctly classified instances (CCI),
# true positive (TP) rate, false positive (FP) rate, false negative (FN) rate, recall,
# specificity, F-measure, false discovery rate (FDR), and Matthews coefficient correlation (MCC)
# were used to evaluate the predictive performances.

import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
from collections import defaultdict

res_dict = defaultdict(list)
# res = []

for pathway_name in pathway_names:
    train_df = pd.read_csv('Datasets/'+ pathway_name+'_train.csv')
    # test_df = pd.read_csv('Datasets/'+ pathway_name+'_test.csv')
    # eval_df = pd.read_csv('Datasets/'+ pathway_name+'_eval.csv')
    
    # print('pathway: ', pathway_name)
    # print(len(train_df[train_df["Class"]=='positive']))
    
    train_df["Class"] = train_df["Class"].apply(lambda x: 1 if x=='positive' else 0)
    # test_df["Class"] = test_df["Class"].apply(lambda x: 1 if x=='positive' else 0)
    # eval_df["Class"] = eval_df["Class"].apply(lambda x: 1 if x=='positive' else 0)
    
    names = ['svc', 'nb', 'dt', 'mlp', 'knn', 'rf', 'sgd']
    models = [BaggingClassifier(SVC()),
              BaggingClassifier(GaussianNB()),
              BaggingClassifier(DecisionTreeClassifier()),
              BaggingClassifier(MLPClassifier()),
              BaggingClassifier(KNeighborsClassifier()),
              BaggingClassifier(RandomForestClassifier()),
              BaggingClassifier(SGDClassifier()),]
    
    # 'shuffle' is added, bc 'random_state' is set
    skf = StratifiedKFold(n_splits=10, random_state=1337, shuffle=True)
    X = train_df.drop('Class', axis=1)
    y = train_df["Class"]
    
    res = []
    for i in range(len(models)):
        fold_num=0
        for train_index, test_index in skf.split(X, y):
            fold_num+=1
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            models[i].fit(X_train, y_train)
            preds = models[i].predict(X_test)
            # print(y_test[y_test[]])
            # res.append(calc_metrics(y_test.tolist(), preds.tolist(), names[i], pathway_name, 'test_df', fold_num))
            res.append(calc_metrics(y_test.tolist(), preds.tolist(), names[i], pathway_name, 'test_df', fold_num))

    res_df = pd.DataFrame(res, columns=["model_name", "pw_name", "dataset_name", "fold_num", "acc", "f1", "mcc", "tp", "fp", "tn", "fn", "recall", "precision", "specifity", "fpr", "tpr", "fnr", "fdr"])
    res_dict[pathway_name] = res_df
    
    file_name = '10fold_results_' + pathway_name + '.csv'
    
    res_df.to_csv(file_name, index=False)
    print(file_name, "is done.")
    
del y_test, y_train, X_test, X_train, X, y

10fold_results_amine_and_polyamine.csv  is done.
10fold_results_amino_acid.csv  is done.


In [58]:
pd.read_csv('10fold_results_amine_and_polyamine.csv').drop('fold_num', axis=1).groupby('model_name').mean()

,acc,f1,mcc,tp,fp,tn,fn,recall,precision,specifity,fpr,tpr,fnr,fdr
model_name,,,,,,,,,,,,,,
dt,0.992531,0.881797,0.881604,15.6,0.9,529.2,3.2,0.947808,0.947808,0.998302,0.001698,0.829532,0.170468,0.052192
knn,0.995810,0.933513,0.934135,16.5,0.0,530.1,2.3,1.000000,1.000000,1.000000,0.000000,0.877485,0.122515,0.000000
mlp,0.993624,0.901186,0.899846,16.3,1.0,529.1,2.5,0.942729,0.942729,0.998114,0.001886,0.866959,0.133041,0.057271
nb,0.845504,0.292756,0.359638,17.2,83.2,446.9,1.6,0.175160,0.175160,0.843050,0.156950,0.914327,0.085673,0.824840
rf,0.993076,0.884055,0.888080,15.0,0.0,530.1,3.8,1.000000,1.000000,1.000000,0.000000,0.796784,0.203216,0.000000
sgd,0.994717,0.916128,0.916771,16.2,0.3,529.8,2.6,0.983224,0.983224,0.999434,0.000566,0.861696,0.138304,0.016776
svc,0.995263,0.923086,0.924770,16.2,0.0,530.1,2.6,1.000000,1.000000,1.000000,0.000000,0.861696,0.138304,0.000000


In [59]:
pd.read_csv('10fold_results_amino_acid.csv').drop('fold_num', axis=1).groupby('model_name').mean()

,acc,f1,mcc,tp,fp,tn,fn,recall,precision,specifity,fpr,tpr,fnr,fdr
model_name,,,,,,,,,,,,,,
dt,0.963200,0.722646,0.727574,26.5,1.6,502.2,18.6,0.943446,0.943446,0.996825,0.003175,0.587391,0.412609,0.056554
knn,0.973038,0.810392,0.806698,32.3,2.0,501.8,12.8,0.942145,0.942145,0.996030,0.003970,0.715894,0.284106,0.057855
mlp,0.972309,0.804793,0.800821,32.1,2.2,501.6,13.0,0.935837,0.935837,0.995634,0.004366,0.711401,0.288599,0.064163
nb,0.460742,0.212557,0.171722,39.8,290.7,213.1,5.3,0.120911,0.120911,0.422993,0.577007,0.882271,0.117729,0.879089
rf,0.962653,0.703954,0.722946,24.6,0.0,503.8,20.5,1.000000,1.000000,1.000000,0.000000,0.545314,0.454686,0.000000
sgd,0.959556,0.692035,0.695799,25.4,2.5,501.3,19.7,0.909598,0.909598,0.995038,0.004962,0.562995,0.437005,0.090402
svc,0.964840,0.726296,0.741009,25.9,0.1,503.7,19.2,0.996296,0.996296,0.999802,0.000198,0.574058,0.425942,0.003704


In [38]:
# res_df.drop('fold_num', axis=1).loc[res_df["pw_name"] == "amine_and_polyamine"].groupby('model_name').mean()

In [65]:
# The best classifiers for each metabolic pathway were ranked using the Kruskal-Wallis test.
# Bu test algoritmaları nasıl sıralıyor anlamadım, benim anladığım fark var mı diye bakıyor dağılımlarda

from scipy.stats import kruskal

# res_rows = res_df.drop('fold_num', axis=1).groupby('model_name').mean()
for pw_name, df in res_dict.items():
    res_rows = df.drop('fold_num', axis=1).groupby('model_name').mean()

    for i in range(len(res_rows.index)):
        for j in range(i+1, len(res_rows.index)):
            print(pw_name, "-", str(res_rows.index[i]), "-", str(res_rows.index[j]))
            stat, p = kruskal(res_rows.loc[str(res_rows.index[i])], res_rows.loc[str(res_rows.index[j])])
            print('Statistics=%.3f, p=%.3f' % (stat, p))
            if kr.pvalue < 0.05:
                print("statistically significant p value is found: ", str(res_rows.index[i], str(res_rows.index[j])))
            print("")

amine_and_polyamine dt knn
Statistics=0.019, p=0.890

amine_and_polyamine dt mlp
Statistics=0.076, p=0.783

amine_and_polyamine dt nb
Statistics=0.414, p=0.520

amine_and_polyamine dt rf
Statistics=0.002, p=0.963

amine_and_polyamine dt sgd
Statistics=0.019, p=0.890

amine_and_polyamine dt svc
Statistics=0.019, p=0.890

amine_and_polyamine knn mlp
Statistics=0.005, p=0.945

amine_and_polyamine knn nb
Statistics=0.135, p=0.713

amine_and_polyamine knn rf
Statistics=0.013, p=0.908

amine_and_polyamine knn sgd
Statistics=0.034, p=0.854

amine_and_polyamine knn svc
Statistics=0.013, p=0.908

amine_and_polyamine mlp nb
Statistics=0.541, p=0.462

amine_and_polyamine mlp rf
Statistics=0.090, p=0.765

amine_and_polyamine mlp sgd
Statistics=0.019, p=0.890

amine_and_polyamine mlp svc
Statistics=0.005, p=0.945

amine_and_polyamine nb rf
Statistics=0.076, p=0.783

amine_and_polyamine nb sgd
Statistics=0.357, p=0.550

amine_and_polyamine nb svc
Statistics=0.135, p=0.713

amine_and_polyamine rf sgd

In [197]:
res_dict["amine_and_polyamine"].drop('fold_num', axis=1).groupby('model_name').mean().loc["nb"]

acc            0.845504
f1             0.292756
mcc            0.359638
tp            17.200000
fp            83.200000
tn           446.900000
fn             1.600000
recall         0.175160
precision      0.175160
specifity      0.843050
fpr            0.156950
tpr            0.914327
fnr            0.085673
fdr            0.824840
Name: nb, dtype: float64

In [198]:
# The best classifiers were pre-selected, seeking a low FP & high F-measure and CCI

from collections import OrderedDict

best_models = {}    # {"pathway": [best 3 models]}

for pw_name, df in res_dict.items():
    df = df.drop('fold_num', axis=1).groupby('model_name').mean()
    
    models = []
    scores = {}
    for i in range(len(df)):
        model = str(res_rows.index[i])
        row = df.loc[model]

        # low FP
        FP = row["fp"]
        
        # high f-measure
        F1 = row["f1"]
        
        # high percentage of correctly classified instances
        CCI = row["acc"]

        score = (CCI + F1) - FP
        scores[model] = score
        
    scores = OrderedDict(sorted(scores.items(), key=lambda kv: kv[1], reverse=True))
    
    i = 0
    for model, score in scores.items():
        if (i == 3):
            break
        
        models.append(model)
        i += 1
    
    best_models[pw_name] = models

In [199]:
scores

OrderedDict([('rf', 1.66660625964676),
             ('svc', 1.5911357528288048),
             ('dt', 0.08584549131292851),
             ('knn', -0.21657021269860977),
             ('mlp', -0.4228980122630448),
             ('sgd', -0.8484098342774564),
             ('nb', -290.0267013095288)])

In [200]:
best_models

{'amine_and_polyamine': ['knn', 'svc', 'rf'],
 'amino_acid': ['rf', 'svc', 'dt']}

In [179]:
# estimators for bagging classifier
bagging_cls = {'svc': ('svc', BaggingClassifier(SVC())),
               'dt': ('dt', BaggingClassifier(DecisionTreeClassifier())),
               'mlp': ('mlp', BaggingClassifier(MLPClassifier())),
               'nb' : ('nb', BaggingClassifier(GaussianNB())),
               'rf': ('rf', BaggingClassifier(RandomForestClassifier())),
               'knn': ('knn', BaggingClassifier(KNeighborsClassifier())),
               'sgd': ('sgd', BaggingClassifier(SGDClassifier()))}

In [190]:
params_rf = {'rf__base_estimator__bootstrap': [True],
             'rf__base_estimator__max_depth': [80, 100]}
# 'rf__base_estimator__max_features': [2, 3]
# 'rf__base_estimator__min_samples_leaf': [3, 4],
# 'rf__base_estimator__min_samples_split': [8, 10],
# 'rf__base_estimator__n_estimators': [100, 200] 

params_SVC={'svc__base_estimator__C': [1, 100, 1000],
            'svc__base_estimator__gamma': [1, 0.01, 0.0001] }
# 'svc__base_estimator__kernel': ['rbf']

params_MLP = {'mlp__base_estimator__hidden_layer_sizes': [(10,30,10),(20,)],
              'mlp__base_estimator__activation': ['tanh', 'relu'] }
# 'mlp__base_estimator__solver': ['sgd', 'adam'],
# 'mlp__base_estimator__alpha': [0.0001, 0.05],
# 'mlp__base_estimator__learning_rate': ['constant','adaptive']

params_DT = { 'dt__base_estimator__max_depth': list(range(1,10)),
              'dt__base_estimator__min_samples_leaf': list(range(1,10,2))}

params_KNN = { 'knn__base_estimator__n_neighbors': list(range(1, 31)) }

params_SGD = {
    'sgd__base_estimator__penalty': ['l2'],
    'sgd__base_estimator__n_jobs': [-1] }
# 'sgd__base_estimator__C': [1e-7, 1e-5, 1e-3]

all_params = {"rf": params_rf, 
              "svc": params_SVC, 
              "mlp": params_MLP, 
              "dt": params_DT, 
              "knn": params_KNN, 
              "sgd": params_SGD}

In [191]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV

In [192]:
voting_models_w_params = {}    # ["pathway": (voting model, parameters)]

for pw_name, models in best_models.items():
    
    estimators = []
    parameters = {}
    
    for model in models:
        # model's estimator
        estimators.append(bagging_cls[model])
        
        # there is no param for NB
        if model in all_params.keys():
            for param, values in all_params[model].items():
                parameters[param] = values
                
    # pramameters = 
        
    voting_model = VotingClassifier(estimators=estimators, voting='soft')
    voting_models_w_params[pw_name] = (voting_model, parameters)

In [193]:
# parameters tuning of combined algorithms

best_scores = {}    # "model": best scores
best_params = {}    # "model": best parameters

for pathway_name in pathway_names:
    df = pd.read_csv('Datasets/' + pathway_name+'_train.csv')
    
    voting_model = voting_models_w_params[pathway_name][0]
    parameters = voting_models_w_params[pathway_name][1]
    
    print("Starting for", pathway_name)
    clf = GridSearchCV(voting_model, parameters, n_jobs=-1, verbose=3, scoring='f1_macro', cv=2)
    clf.fit(df.drop('Class', axis=1), df["Class"])
    
    best_scores[pathway_name] = clf.best_score_
    best_params[pathway_name] = clf.best_params_

    print(clf.best_score_)
    print(clf.best_params_)

Starting for amine_and_polyamine
Fitting 2 folds for each of 180 candidates, totalling 360 fits
0.9368153506734713
{'dt__base_estimator__max_depth': 5, 'dt__base_estimator__min_samples_leaf': 9, 'mlp__base_estimator__activation': 'relu', 'mlp__base_estimator__hidden_layer_sizes': (20,)}
Starting for amino_acid
Fitting 2 folds for each of 4 candidates, totalling 8 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multi

0.6855790501839225
{'mlp__base_estimator__activation': 'tanh', 'mlp__base_estimator__hidden_layer_sizes': (20,), 'sgd__base_estimator__n_jobs': -1, 'sgd__base_estimator__penalty': 'l2'}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [194]:
best_scores

{'amine_and_polyamine': 0.9368153506734713, 'amino_acid': 0.6855790501839225}

In [195]:
best_params

{'amine_and_polyamine': {'dt__base_estimator__max_depth': 5,
  'dt__base_estimator__min_samples_leaf': 9,
  'mlp__base_estimator__activation': 'relu',
  'mlp__base_estimator__hidden_layer_sizes': (20,)},
 'amino_acid': {'mlp__base_estimator__activation': 'tanh',
  'mlp__base_estimator__hidden_layer_sizes': (20,),
  'sgd__base_estimator__n_jobs': -1,
  'sgd__base_estimator__penalty': 'l2'}}